In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import jensenshannon, jaccard
import pandas as pd
import preprocess
from itertools import combinations

In [52]:
majority_path = r'C:\Users\IsmailKaraman\workspace\data\privacy_policy_data\OPP-115_v2\majority.csv'
df = pd.read_csv(majority_path)
X = df['text'].apply(preprocess.preprocess_text)
y = df.drop(['text'], axis=1)

In [53]:
vectorizer = TfidfVectorizer()
X = pd.DataFrame(vectorizer.fit_transform(X).toarray())

In [58]:
similarities = {}

In [59]:
def vector_similarity(p, q):

    return jensenshannon(p, q)

In [60]:
def calculate_within_class_similarity(vecs):
    
    similarities = []
    
    for i,j in list(combinations(vecs.index, 2)):
        similarities.append(vector_similarity(vecs.loc[i], vecs.loc[j]))                
    try:
        avg_similarity = sum(similarities)/len(similarities)
    except AssertionErrors:
        print('Error occured')
        
    return avg_similarity 

In [61]:
def calculate_between_class_similarities(col1, col2):
    
    similarities = []
    
    for idx1 in y[y[col1]==1].index:
        for idx2 in y[y[col2]==1].index:
            similarities.append(vector_similarity(X.loc[idx1], X.loc[idx2]))
    
    return sum(similarities)/len(similarities)       

In [62]:
for label in y.columns:
    
    indexes = y[y[label]==1].index
    similarities[(label, label)] = calculate_within_class_similarity(X.loc[indexes])

KeyboardInterrupt: 

In [ ]:
for col1, col2 in list(combinations(y.columns, 2)):
    similarities[(col1, col2)] = calculate_between_class_similarities(col1, col2)

In [ ]:
sim_df = pd.DataFrame(index=y.columns, columns=y.columns)
for (col1,col2), sim in similarities.items():
    sim_df.loc[col1, col2] =  float(sim)

In [ ]:
import seaborn as sns

sns.heatmap(sim_df.fillna(0), annot=True,
        xticklabels=sim_df.columns,
        yticklabels=sim_df.columns, cmap="BuPu")